In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from setup_module.estimators_get_pipe import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


Using MPS


0it [00:00, ?it/s]

Using MPS


<Figure size 640x480 with 0 Axes>

### Set variables

In [3]:
# Variables
t = time.time()
n_jobs = -1
n_splits = 10
n_repeats = 3
random_state = 42
refit = True
class_weight = 'balanced'
cv = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
)
scoring = 'recall'
scores = [
    'recall', 'accuracy', 'f1', 'roc_auc',
    'explained_variance', 'matthews_corrcoef'
]
scorers = {
    'precision_score': make_scorer(precision_score, zero_division=0),
    'recall_score': make_scorer(recall_score, zero_division=0),
    'accuracy_score': make_scorer(accuracy_score, zero_division=0),
}
protocol = pickle.HIGHEST_PROTOCOL
analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'
classified_columns = ['Warmth_Probability', 'Competence_Probability']
metrics_dict = {
    # f'{scoring.title()} Best Score': np.nan,
    # f'{scoring.title()} Best Threshold': np.nan,
    # 'Train - Mean Cross Validation Score': np.nan,
    # f'Train - Mean Cross Validation - {scoring.title()}': np.nan,
    # f'Train - Mean Explained Variance - {scoring.title()}': np.nan,
    # 'Test - Mean Cross Validation Score': np.nan,
    # f'Test - Mean Cross Validation - {scoring.title()}': np.nan,
    # f'Test - Mean Explained Variance - {scoring.title()}': np.nan,
    'Explained Variance': np.nan,
    'Accuracy': np.nan,
    'Balanced Accuracy': np.nan,
    'Precision': np.nan,
    'Average Precision': np.nan,
    'Recall': np.nan,
    'F1-score': np.nan,
    'Matthews Correlation Coefficient': np.nan,
    'Brier Score': np.nan,
    'Fowlkes–Mallows Index': np.nan,
    'R2 Score': np.nan,
    'ROC': np.nan,
    'AUC': np.nan,
    'Log Loss/Cross Entropy': np.nan,
    'Cohen’s Kappa': np.nan,
    'Geometric Mean': np.nan,
    'Classification Report': np.nan,
    'Imbalanced Classification Report': np.nan,
    'Confusion Matrix': np.nan,
    'Normalized Confusion Matrix': np.nan,
}

# Transformer variables
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available(
) else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
# Set random seed
random_state = 42
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
cores = multiprocessing.cpu_count()
torch.Generator(device_name).manual_seed(random_state)
cores = multiprocessing.cpu_count()
accelerator = Accelerator()
torch.autograd.set_detect_anomaly(True)
os.environ.get('TOKENIZERS_PARALLELISM')
os.environ.get('PYTORCH_MPS_HIGH_WATERMARK_RATIO')
os.environ.get('TRANSFORMERS_CACHE')
openai_token = os.environ['OPENAI_API_KEY']
huggingface_token = os.environ['HUGGINGFACE_API_KEY']
# load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_quant_type='nf4'
quantization_config_dict = {
    'load_in_8bit': True,
    'llm_int8_skip_modules': ['lm_head'],
}
skip_fitted_estimators = False
evaluate_estimator_on_concat = False
hyperparameter_tuning = False


Using MPS


# Functions


In [4]:
def load_classified_df(
    df, df_name, df_len, done_dfs_name, df_save_dir
):
    print(f'Loading {df_name}_{done_dfs_name}...')
    df = pd.read_pickle(f'{df_save_dir}{df_name}_{done_dfs_name}.pkl')
    print(f'Dataframe {df_name}_{done_dfs_name} loaded with shape: {df.shape}')

    return df


In [5]:
def extract_prediction(text):
    if pred := classifier(text):
        return pd.Series([pred[0]['label'].split('LABEL_')[1], pred[0]['score']])
    else:
        return None, None


In [6]:
class ToDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # check is encodings and labels are tensors
        for key, val in self.encodings.items():
            if not torch.is_tensor(val[idx]):
                self.encodings[key][idx] = torch.tensor(val[idx], dtype=torch.long, device=device)
        if self.labels is not None and not torch.is_tensor(self.labels[idx]):
            with contextlib.suppress(ValueError):
                self.labels[idx] = torch.tensor(self.labels[idx], dtype=torch.long, device=device)
        item = {key: val[idx].to(device) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


In [7]:
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = self._calculate_class_weights(self.train_dataset)

    def _calculate_class_weights(self, dataset):
        # Count the number of samples in each class
        class_counts = torch.bincount(torch.tensor(dataset.labels, device=device))

        # Calculate weight and pos_weight
        num_negative = class_counts[0].item()
        num_positive = class_counts[1].item()
        # weight_neg = num_positive / (num_negative + 1e-5)
        weight_pos = num_negative / (num_positive + 1e-5)
        # weight = torch.tensor([weight_neg, weight_pos], device=device)
        pos_weight = torch.tensor([weight_pos], device=device)

        return pos_weight

    def _calculate_calibration_loss(self, logits, labels):
        return torch.nn.BCEWithLogitsLoss(pos_weight=self.pos_weight.to(device))(
            logits.to(device), torch.nn.functional.one_hot(labels, logits.size(-1)).long().float().to(device)
        )

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop('labels').to(device)
        outputs = model(**inputs)
        logits = outputs.get('logits').to(device)
        loss = self._calculate_calibration_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [8]:
# Function to get y_pred and y_pred_prob
def preprocess_logits_for_metrics_from_logits(y_pred_logits, print_enabled=None):

    if print_enabled is None:
        print_enabled = True

    # Get y_pred
    if not torch.is_tensor(y_pred_logits):
        y_pred_logits_tensor = torch.tensor(y_pred_logits, device=device)
    if print_enabled:
        print('-'*20)
        print('Getting y_pred through argmax of y_pred_logits...')
    try:
        y_pred_array = torch.argmax(y_pred_logits_tensor, axis=-1)
        if print_enabled: print('Using torch.argmax.')
    except Exception:
        y_pred_array = y_pred_logits.argmax(axis=-1)
        if print_enabled: print('Using np.argmax.')
    if print_enabled:
        print(f'y_pred_array shape: {y_pred_array.shape}')
        print('-'*20)
        print('Flattening y_pred...')
    y_pred = y_pred_array.flatten().tolist()
    if print_enabled:
        print(f'y_pred length: {len(y_pred)}')
        print('-'*20)

    # Get y_pred_prob
    if print_enabled:
        print('-'*20)
        print('Getting y_pred_prob through sigmoid of y_pred_logits...')
    try:
        y_pred_prob_array = torch.nn.functional.sigmoid(y_pred_logits_tensor)
        if print_enabled: print('Using torch.nn.functional.sigmoid.')
    except Exception:
        y_pred_prob_array = scipy.special.expit(y_pred_logits_tensor, axis=-1)
        if print_enabled: print('Using scipy.special.expit.')
    # from: https://discuss.huggingface.co/t/different-results-predicting-from-trainer-and-model/12922
    assert all(y_pred_prob_array.argmax(axis=-1) == y_pred_array), 'Argmax of y_pred_prob_array does not match y_pred_array.'
    y_pred_prob = y_pred_prob_array[:, -1].flatten().tolist()
    if print_enabled:
        print(f'y_pred_prob shape: {y_pred_prob_array.shape}')
        print('-'*20)
        print('Flattening y_pred_prob and extracting probabilities of 1...')
        print(f'y_pred length: {len(y_pred_prob)}')
        print('-'*20)

    return (
        y_pred_array, y_pred, y_pred_prob_array, y_pred_prob
    )


In [9]:
def prob_confirmatory_tests(y_pred, y_pred_prob):

    # Confirmatory Regression
    print('+'*20)
    print('Confirmatory Tests validating the linear relationship between y_pred and y_pred_prob')
    print('-'*20)
    print('T-Test y_pred_prob ~ y_pred:')
    levene = scipy.stats.levene(y_pred_prob, y_pred)
    equal_var_levene = levene.pvalue < 0.05
    print(scipy.stats.ttest_ind(y_pred_prob, y_pred, equal_var=equal_var_levene))

    print('\n')
    print('-'*20)
    print('Logit y_pred ~ y_pred_prob:')
    try:
        logit_model = sm.Logit(endog=y_pred, exog=y_pred_prob)
        logit_results = logit_model.fit()
        std_coef = logit_results.params[0] / np.std(y_pred_prob)
        std_err = logit_results.bse[0]
        log_likelihood = logit_results.llf
        print(logit_results.summary())
        print('-'*20)
        print(f'Std Coef: {std_coef}')
        print(f'Std Err: {std_err}')
        print(f'Log Likelihood: {log_likelihood}')
    except Exception as e:
        print(type(e).__name__)

    print('-'*20)
    print('\n')
    print('-'*20)
    print('OLS y_pred_prob ~ y_pred:')
    try:
        ols_model = sm.OLS(endog=y_pred_prob, exog=y_pred)
        ols_results = ols_model.fit()
        std_coef = ols_results.params[0] / np.std(y_pred)
        std_err = ols_results.bse[0]
        print(ols_results.summary())
        print('-'*20)
        print(f'Std Coef: {std_coef}')
        print(f'Std Err: {std_err}')
    except Exception as e:
        print(type(e).__name__)

    print('-'*20)
    print('+'*20)
    print('\n')


# Classifying

### READ DATA

In [10]:
# # ATTN: IF THIS IS THE FIRST TIME YOU ARE CLASSIFYING JOBS, UNCOMMENT AND RUN THIS CODE
# with open(f'{data_dir}df_jobs_len.txt', 'r') as f:
#     df_jobs_len = int(f.read())
# df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_classification.pkl')
# assert len(df_jobs) == df_jobs_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'
# print(f'Dataframe df_jobs loaded with shape: {df_jobs.shape}')
# with open(f'{data_dir}df_manual_len.txt', 'r') as f:
#     df_manual_len = int(f.read())
# df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_training.pkl')
# assert len(df_manual) == df_manual_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'
# print(f'Dataframe df_manual loaded with shape: {df_manual.shape}')


In [11]:
with open(f'{data_dir}df_jobs_len.txt', 'r') as f:
    df_jobs_len = int(f.read())

with open(f'{data_dir}df_manual_len.txt', 'r') as f:
    df_manual_len = int(f.read())

for done_dfs_name in ['classified', 'classified_Warmth_Competence', 'classified_Warmth']:
    if (os.path.exists(f'{df_save_dir}df_jobs_{done_dfs_name}.pkl') and os.path.getsize(f'{df_save_dir}df_jobs_{done_dfs_name}.pkl') > 0) and (os.path.exists(f'{df_save_dir}df_manual_{done_dfs_name}.pkl') and os.path.getsize(f'{df_save_dir}df_manual_{done_dfs_name}.pkl') > 0):

        df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_{done_dfs_name}.pkl')
        assert len(df_jobs) == df_jobs_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'
        df_manual = pd.read_pickle(f'{df_save_dir}df_manual_{done_dfs_name}.pkl')

        if (
            done_dfs_name == 'classified'
            and 'Warmth' in df_jobs.columns
            and 'Warmth_Probability' in df_jobs.columns
            and 'Competence' in df_jobs.columns
            and 'Competence_Probability' in df_jobs.columns
        ):
            df_jobs = load_classified_df(df_jobs, 'df_jobs', df_jobs_len, done_dfs_name, df_save_dir)
            df_manual = load_classified_df(df_manual, 'df_manual', df_manual_len, done_dfs_name, df_save_dir)
            break

        elif (
            done_dfs_name == 'classified_Warmth_Competence'
            and 'Warmth' in df_jobs.columns
            and 'Warmth_Probability' in df_jobs.columns
            and 'Competence' in df_jobs.columns
            and 'Competence_Probability' in df_jobs.columns
        ):
            df_jobs = load_classified_df(df_jobs, 'df_jobs', df_jobs_len, done_dfs_name, df_save_dir)
            df_manual = load_classified_df(df_manual, 'df_manual', df_manual_len, done_dfs_name, df_save_dir)
            break

        elif (
            done_dfs_name == 'classified_Warmth'
            and 'Warmth' in df_jobs.columns
            and 'Warmth_Probability' in df_jobs.columns
            and 'Competence' not in df_jobs.columns
            and 'Competence_Probability' not in df_jobs.columns
        ):
            df_jobs = load_classified_df(df_jobs, 'df_jobs', df_jobs_len, done_dfs_name, df_save_dir)
            df_manual = load_classified_df(df_manual, 'df_manual', df_manual_len, done_dfs_name, df_save_dir)
            break

else:
    with open(f'{data_dir}df_jobs_classified_len.txt', 'r') as f:
        df_jobs_len = int(f.read())

    with open(f'{data_dir}df_manual_classified_len.txt', 'r') as f:
        df_manual_len = int(f.read())

    print('Loading df_jobs_for_classification...')
    df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_classification.pkl')
    assert len(df_jobs) == df_jobs_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'
    print(f'Dataframe df_jobs_for_classification loaded with shape: {df_jobs.shape}')
    df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_training.pkl')
    assert len(df_manual) == df_manual_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'
    print(f'Dataframe df_manual_for_training loaded with shape: {df_manual.shape}')


Loading df_jobs_classified...
Dataframe df_jobs_classified loaded with shape: (307154, 95)
Loading df_manual_classified...
Dataframe df_manual_classified loaded with shape: (5358, 79)


In [12]:
%%time
print('#'*40)
print('Starting!')
print('#'*40)

done_cols = ''
warmth_classifier = 'GPT2ForSequenceClassification'
competence_classifier = 'BertForSequenceClassification'
final_estimators_dict = {
    'Warmth': {
        'classifier_name': warmth_classifier,
        'vectorizer_name': ''.join(transformers_pipe[warmth_classifier]["model_name"].split('-')).upper(),
    },
    'Competence': {
        'classifier_name': competence_classifier,
        'vectorizer_name': ''.join(transformers_pipe[competence_classifier]["model_name"].split('-')).upper(),
    },
}

for col in tqdm.tqdm(analysis_columns):
    if col not in df_jobs.columns and f'{col}_predicted' not in df_manual.columns:
        print('-'*20)
        final_estimators_dict[col]['path_suffix'] = path_suffix = f' - {col} - {(vectorizer_name := final_estimators_dict[col]["vectorizer_name"])} + {(classifier_name := final_estimators_dict[col]["classifier_name"])} (Save_protocol={protocol})'

        if classifier_name in list(classifiers_pipe.keys()):
            method = 'Supervised'
            with open(f'{data_dir}{method}_results_save_path.txt', 'r') as f:
                results_save_path = f.read()
            print('-'*20)
            print(f'Using {classifier_name} from {method} pipeline.')
            print('Loading Supervised Estimator.')
            with open(
                f'{results_save_path}{method} Fitted Estimator {path_suffix}.pkl', 'rb'
            ) as f:
                estimator = joblib.load(f)
            print('Done loading Supervised Estimator!')

            print('-'*20)
            print('Classifying data.')
            # df_jobs
            print('Classifying df_jobs.')
            X = np.array(list(df_jobs[text_col].astype('str').values))
            df_jobs[col] = estimator.predict(X)
            if hasattr(estimator, 'predict_proba'):
                # Get the the whole of the last column, which is the  probability of 1, and flatten to list
                df_jobs[f'{col}_Probability'] = estimator.predict_proba(X)[:, -1]
            # df_manual
            print('Classifying df_manual to generate instrumental variables.')
            X_instrument = np.array(list(df_manual[text_col].astype('str').values))
            df_manual[f'{col}_predicted'] = estimator.predict(X_instrument)
            if hasattr(estimator, 'predict_proba'):
                # Get the the whole of the last column, which is the  probability of 1, and flatten to list
                df_manual[f'{col}_Probability_predicted'] = estimator.predict_proba(X_instrument)[:, -1]

            print(f'Done classifying data using {classifier_name} for {col}!')
            print('-'*20)

        elif classifier_name in list(transformers_pipe.keys()):
            method = 'Transformers'
            with open(f'{data_dir}{method}_results_save_path.txt', 'r') as f:
                results_save_path = f.read().strip('\n')
            with open(f'{data_dir}{method}_done_xy_save_path.txt', 'r') as f:
                done_xy_save_path = f.read().strip('\n')
            with open(f'{done_xy_save_path}{method} training_args_dict - {col} - {vectorizer_name} + {classifier_name}.json', 'r') as f:
                training_args_dict = json.load(f)

            print('-'*20)
            print(f'Using {classifier_name} from {method} pipeline.')
            model = transformers_pipe[classifier_name]['model']
            tokenizer = transformers_pipe[classifier_name]['tokenizer']
            config = transformers_pipe[classifier_name]['config']

            print(f'Loading Fitted Transformer {classifier_name} from pretrained.')
            estimator_dir = f'{results_save_path}{method} Fitted Estimator{path_suffix}.model'
            fitted_estimator = model.from_pretrained(estimator_dir, trust_remote_code=True)
            if hasattr(fitted_estimator, 'to'):
                fitted_estimator = fitted_estimator.to(device)
            tokenizer = tokenizer.from_pretrained(estimator_dir, trust_remote_code=True)
            config = config.from_pretrained(f'{estimator_dir}/config.json', trust_remote_code=True)
            print(f'Done loading Fitted Transformer {classifier_name} from pretrained!')

            # Tokenize df_jobs
            X = df_jobs[text_col].astype('str').values.tolist()
            encodings = tokenizer(
            X, truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
            ).to(device)
            dataset = ToDataset(encodings)

            # Tokenize df_manual
            X_instrument = df_manual[text_col].astype('str').values.tolist()
            encodings_instrument = tokenizer(
            X_instrument, truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
            ).to(device)
            dataset_instrument = ToDataset(encodings_instrument)

            # Accelerate model
            (
                fitted_estimator, tokenizer, dataset, dataset_instrument
            ) = accelerator.prepare(
                fitted_estimator, tokenizer, dataset, dataset_instrument
            )

            print(f'Getting estimator for {col}.')
            estimator = Trainer(
                model=fitted_estimator,
                tokenizer=tokenizer,
                args=TrainingArguments(**training_args_dict),
            )
            if estimator.place_model_on_device:
                estimator.model.to(device)

            # df_jobs
            print('-'*20)
            print(f'Classifying data using {classifier_name} for {col}.')
            print('Classifying df_jobs.')
            (y_pred_logits, y_labels, metrics) = estimator.predict(dataset)
            y_pred_array, y_pred, y_pred_prob_array, y_pred_prob = preprocess_logits_for_metrics_from_logits(y_pred_logits)

            # Assign to dataframe
            df_jobs[col] = y_pred
            df_jobs[f'{col}_Probability'] = y_pred_prob
            df_jobs[col] = df_jobs[col].astype(int)
            df_jobs[f'{col}_Probability'] = df_jobs[f'{col}_Probability'].astype(float)

            # df_manual
            print('Classifying df_manual to generate instrumental variables.')
            (y_pred_logits_instrument, y_labels_instrument, metrics_instrument) = estimator.predict(dataset_instrument)
            y_pred_array_instrument, y_pred_instrument, y_pred_prob_array_instrument, y_pred_prob_instrument = preprocess_logits_for_metrics_from_logits(y_pred_logits_instrument)

            # Assign to dataframe
            df_manual[f'{col}_predicted'] = y_pred_instrument
            df_manual[f'{col}_Probability_predicted'] = y_pred_prob_instrument
            df_manual[f'{col}_predicted'] = df_manual[f'{col}_predicted'].astype(int)
            df_manual[f'{col}_Probability_predicted'] = df_manual[f'{col}_Probability_predicted'].astype(float)

            print(f'Done classifying data using {classifier_name} for {col}!')
            print('-'*20)

        done_cols += f'_{col}'
        assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
        # df_jobs
        df_jobs.to_pickle(f'{df_save_dir}df_jobs_classified{done_cols}.pkl')
        df_jobs.to_csv(f'{df_save_dir}df_jobs_classified{done_cols}.csv', index=False)
        # df_manual
        df_manual.to_pickle(f'{df_save_dir}df_manual_classified{done_cols}.pkl')
        df_manual.to_csv(f'{df_save_dir}df_manual_classified{done_cols}.csv', index=False)
    else:
        print('-'*20)
        print(f'Column {col} already exists in dataframe. Skipping.')
        print('-'*20)
    # Confirmatory Regression
    print('='*20)
    print(f'Confirmatory test for df_jobs {col}')
    prob_confirmatory_tests(df_jobs[col].values, df_jobs[f'{col}_Probability'].values)
    print(f'Confirmatory test for df_manual {col}')
    prob_confirmatory_tests(df_manual[col], df_manual[f'{col}_predicted'].values)
    print('='*20)


########################################
Starting!
########################################


  0%|          | 0/2 [00:00<?, ?it/s]

--------------------
Column Warmth already exists in dataframe. Skipping.
--------------------
Confirmatory test for df_jobs Warmth
++++++++++++++++++++
Confirmatory Tests validating the linear relationship between y_pred and y_pred_prob
--------------------
T-Test y_pred_prob ~ y_pred:
TtestResult(statistic=85.36721885824605, pvalue=0.0, df=614306.0)


--------------------
Logit y_pred ~ y_pred_prob:
Optimization terminated successfully.
         Current function value: 0.670171
         Iterations 5


 50%|█████     | 1/2 [00:00<00:00,  1.02it/s]

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:               307154
Model:                          Logit   Df Residuals:                   307153
Method:                           MLE   Df Model:                            0
Date:                Thu, 16 Nov 2023   Pseudo R-squ.:                 -0.2154
Time:                        04:20:56   Log-Likelihood:            -2.0585e+05
converged:                       True   LL-Null:                   -1.6936e+05
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9054      0.008    114.584      0.000       0.890       0.921
--------------------
Std Coef: 2.471884596757334
Std Err: 0.007901333384493541
Log Likelihood: -205845.82757131173
----------------

100%|██████████| 2/2 [00:02<00:00,  1.09s/it]

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:               307154
Model:                          Logit   Df Residuals:                   307153
Method:                           MLE   Df Model:                            0
Date:                Thu, 16 Nov 2023   Pseudo R-squ.:                  0.2690
Time:                        04:20:57   Log-Likelihood:            -1.5535e+05
converged:                       True   LL-Null:                   -2.1252e+05
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             2.6697      0.010    262.497      0.000       2.650       2.690
--------------------
Std Coef: 7.061947974069714
Std Err: 0.010170227567633736
Log Likelihood: -155346.88954043572
----------------

## Inspect classified data

In [13]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame) and all(col in df_jobs.columns for col in dvs_all), f'ERORR: LENGTH OF DF = {len(df_jobs)}'

df_jobs.to_pickle(f'{df_save_dir}df_jobs_classified.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_classified.csv', index=False)

with open(f'{data_dir}df_jobs_classified_len.txt', 'w') as f:
    f.write(str(len(df_jobs)))


In [14]:
with open(f'{data_dir}df_jobs_classified_len.txt', 'r') as f:
    df_jobs_len = int(f.read())

df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_classified.pkl')
assert len(df_jobs) == df_jobs_len and all(col in df_jobs.columns for col in dvs_all), f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'


In [15]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame) and all(col in df_manual.columns for col in dvs_all_predicted), f'ERORR: LENGTH OF DF = {len(df_manual)}'

df_manual.to_pickle(f'{df_save_dir}df_manual_classified.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_classified.csv', index=False)

with open(f'{data_dir}df_manual_classified_len.txt', 'w') as f:
    f.write(str(len(df_manual)))


In [16]:
with open(f'{data_dir}df_manual_classified_len.txt', 'r') as f:
    df_manual_len = int(f.read())

df_manual = pd.read_pickle(f'{df_save_dir}df_manual_classified.pkl')
assert len(df_manual) == df_manual_len and all(col in df_manual.columns for col in dvs_all_predicted), f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'


In [17]:
df_jobs.head()


,Search Keyword,Platform,Job ID,Job Title,Company Name,Location,Job Description,Rating,Employment Type,Company URL,Job URL,Job Age,Job Age Number,Collection Date,Data Row,Tracking ID,Industry,Job Date,Type of ownership,Language,Dutch Requirement in Job Ad,English Requirement in Job Ad,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Sector Code,Sector,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Sector Job Advertisement Count,Sector Gender Designation Job Advertisement Count,Sector Age Designation Job Advertisement Count,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description_num_words,Job Description_num_unique_words,Job Description_num_chars,Job Description_num_chars_no_whitespact_and_punt,Job Description_num_punctuations,Job Description spacy_sentencized_lower,Dutch Requirement in Sentence,English Requirement in Sentence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Job Description spacy_tokenized,Job Description spacy_sentencized_cleaned,Job Description nltk_tokenized,Job Description gensim_tokenized,Job Description bert_tokenized,Warmth,Warmth_Probability,Competence,Competence_Probability
0,wholesale,Indeed,pj_da9f2c12243d7031,Transaction Monitoring Expert,Michael Page,Amsterdam,About Our Client\nThe Global KYC organisation ...,-1.00,-1,https://indeed.nl/rc/clk?jk=da9f2c12243d7031&f...,https://nl.indeed.com/vacature-bekijken/pagead...,2 dagen geleden,2 dagen geleden,2021-01-24,NaN,NaN,NaN,NaN,NaN,en,No,No,1,0,1,0,G,Commercial services,11.00,3421.00,43.13,28.47,13.54,4510.00,56.87,34.04,17.85,Mixed Gender,2704.00,34.09,25.44,10.70,5228.00,65.92,35.73,20.69,Mixed Age,7931.00,31.39,1787,6664,11464,0,0,1,1,0,0,1,1,1470.63,2843.37,1938.77,3748.49,1,0,1,0,About Our Client,3,3,16,14,0,558,320,3876,3240,23,about our client,No,No,1,0,1,0,"[about, our, client]",about our client,[client],[client],"[about, our, client]",0,0.01,0,0.02
1,wholesale,Indeed,pj_da9f2c12243d7031,Transaction Monitoring Expert,Michael Page,Amsterdam,About Our Client\nThe Global KYC organisation ...,-1.00,-1,https://indeed.nl/rc/clk?jk=da9f2c12243d7031&f...,https://nl.indeed.com/vacature-bekijken/pagead...,2 dagen geleden,2 dagen geleden,2021-01-24,NaN,NaN,NaN,NaN,NaN,en,No,No,1,0,1,0,G,Commercial services,11.00,3421.00,43.13,28.47,13.54,4510.00,56.87,34.04,17.85,Mixed Gender,2704.00,34.09,25.44,10.70,5228.00,65.92,35.73,20.69,Mixed Age,7931.00,31.39,1787,6664,11464,0,0,1,1,0,0,1,1,1470.63,2843.37,1938.77,3748.49,1,0,1,0,The Global KYC organisation is part of ING's C...,10,10,56,45,1,558,320,3876,3240,23,the global kyc organisation is part of ing's c...,No,No,1,0,1,0,"[the, global, kyc, organisation, is, part, of,...",the global kyc organisation is part of ing 's ...,"[global, kyc, organisation, part, ing, 's, coo...","[global, kyc, organis, ing, coo, domain]","[the, global, ky, ##c, organisation, is, part,...",0,0.01,0,0.06
2,wholes

In [18]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307154 entries, 0 to 307153
Data columns (total 95 columns):
 #   Column                                                              Non-Null Count   Dtype   
---  ------                                                              --------------   -----   
 0   Search Keyword                                                      307154 non-null  object  
 1   Platform                                                            307154 non-null  object  
 2   Job ID                                                              307154 non-null  object  
 3   Job Title                                                           307154 non-null  object  
 4   Company Name                                                        307149 non-null  object  
 5   Location                                                            307154 non-null  object  
 6   Job Description                                                     307154 non-null  object 

In [19]:
df_jobs.describe()


,Rating,Data Row,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Sector_n,% Sector per Workforce,Sector Job Advertisement Count,Sector Gender Designation Job Advertisement Count,Sector Age Designation Job Advertisement Count,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description_num_words,Job Description_num_unique_words,Job Description_num_chars,Job Description_num_chars_no_whitespact_and_punt,Job Description_num_punctuations,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Warmth,Warmth_Probability,Competence,Competence_Probability
count,165831.00,141182.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00
mean,-0.38,359.68,0.97,0.03,0.94,0.06,6.81,656.79,45.36,5.47,2.60,712.05,54.60,5.37,2.82,528.39,40.86,4.97,2.09,840.04,59.04,5.74,3.32,1369.08,5.42,1432.49,6088.90,8502.33,0.25,0.37,0.38,0.64,0.20,0.15,1.11,0.95,1846.01,2681.76,2238.91,3219.13,0.62,0.46,0.45,0.08,17.67,15.81,114.11,95.86,0.31,613.29,316.26,4089.23,3387.65,21.97,0.97,0.03,0.94,0.06,0.24,0.33,0.48,0.45
std,1.63,284.66,0.17,0.17,0.23,0.23,2.88,1047.82,19.49,8.72,4.15,1235.51,19.52,9.32,4.89,796.79,10.12,7.49,3.15,1437.93,9.98,9.83,5.69,2216.35,8.77,768.93,1253.12,3997.39,0.44,0.48,0.49,0.48,0.40,0.36,0.78,0.60,974.24,1138.22,1110.80,1207.73,0.64,0.50,0.50,0.28,16.45,12.77,107.75,90.37,0.53,524.43,147.74,3623.96,3021.22,34.37,0.17,0.17,0.23,0.23,0.43,0.37,0.50,0.38
min,-1.00,1.00,0.00,0.00,0.00,0.00,1.00,7.00,12.50,0.06,0.03,21.00,15.63,0.16,0.08,15.00,18.94,0.14,0.06,13.00,44.44,0.09,0.05,29.00,0.11,150.00,3955.00,2351.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,595.24,662.20,721.04,839.94,0.00,0.00,0.00,0.00,3.00,2.00,6.00,1.00,0.00,4.00,4.00,31.00,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01
25%,-1.00,111.00,1.00,0.00,1.00,0.00,4.00,87.00,27.59,0.72,0.34,189.00,34.87,1.43,0.75,95.00,32.76,0.89,0.38,195.00,51.18,1.33,0.77,290.00,1.15,1064.00,3955.00,3778.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1092.89,1854.93,1627.74,1936.98,0.00,0.00,0.00,0.00,7.00,7.00,45.00,38.00,0.00,353.00,213.00,2289.00,1891.00,0.00,1.00,0.00,1.00,0.00,0.00,0.02,0.00,0.04
50%,-1.00,291.00,1.00,0.00,1.00,0.00,7.00,226.00,43.13,1.88,0.89,210.00,56.87,1.58,0.83,205.00,41.67,1.93,0.81,288.00,56.82,1.97,1.14,398.00,1.57,1411.00,6664.00,11464.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,1.00,1366.56,2843.37,1938.77,3571.64,1.00,0.00,0.00,0.00,13.00,12.00,84

In [20]:
df_jobs[['Job ID', 'Job Description', 'Job Description spacy_sentencized']].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307154 entries, 0 to 307153
Data columns (total 3 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Job ID                             307154 non-null  object
 1   Job Description                    307154 non-null  object
 2   Job Description spacy_sentencized  307154 non-null  object
dtypes: object(3)
memory usage: 7.0+ MB


In [21]:
df_jobs[['Job ID', 'Job Description', 'Job Description spacy_sentencized']].head()


,Job ID,Job Description,Job Description spacy_sentencized
0,pj_da9f2c12243d7031,About Our Client\nThe Global KYC organisation ...,About Our Client
1,pj_da9f2c12243d7031,About Our Client\nThe Global KYC organisation ...,The Global KYC organisation is part of ING's C...
2,pj_da9f2c12243d7031,About Our Client\nThe Global KYC organisation ...,Its purpose is Enabling people and organisatio...
3,pj_da9f2c12243d7031,About Our Client\nThe Global KYC organisation ...,Our Global KYC organisation is a first line of...
4,pj_da9f2c12243d7031,About Our Client\nThe Global KYC organisation ...,In our Global KYC organisation you will be wor...


In [22]:
df_jobs = df_jobs.drop_duplicates(subset=['Job ID', 'Job Description', 'Job Description spacy_sentencized']).reset_index(drop=True)
df_jobs = df_jobs.dropna(subset=['Job ID', 'Job Description', 'Job Description spacy_sentencized'] + dvs_all).reset_index(drop=True)
df_jobs = df_jobs.loc[
    (df_jobs['Job Description spacy_sentencized'].progress_apply(lambda x: len(x) > 5))
    & (df_jobs['Job Description_num_words'] > 2)
    & (df_jobs['Job Description_num_chars'] > 5)
    & (df_jobs['Job Description'].progress_apply(lambda x: len(x) > 5))
    & (df_jobs['Job Description spacy_sentencized_num_words'] > 2)
    & (df_jobs['Job Description spacy_sentencized_num_chars'] > 5)
].reset_index(drop=True)


In [23]:
df_jobs['Job Description_num_words'].min()


4

In [24]:
df_jobs['Job Description_num_words'].max()


10385

In [25]:
df_jobs['Job Description spacy_sentencized_num_words'].min()


3

In [26]:
df_jobs['Job Description spacy_sentencized_num_words'].max()


349

In [27]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307154 entries, 0 to 307153
Data columns (total 95 columns):
 #   Column                                                              Non-Null Count   Dtype   
---  ------                                                              --------------   -----   
 0   Search Keyword                                                      307154 non-null  object  
 1   Platform                                                            307154 non-null  object  
 2   Job ID                                                              307154 non-null  object  
 3   Job Title                                                           307154 non-null  object  
 4   Company Name                                                        307149 non-null  object  
 5   Location                                                            307154 non-null  object  
 6   Job Description                                                     307154 non-null  object 

In [28]:
df_jobs.describe()


,Rating,Data Row,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Sector_n,% Sector per Workforce,Sector Job Advertisement Count,Sector Gender Designation Job Advertisement Count,Sector Age Designation Job Advertisement Count,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description_num_words,Job Description_num_unique_words,Job Description_num_chars,Job Description_num_chars_no_whitespact_and_punt,Job Description_num_punctuations,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Warmth,Warmth_Probability,Competence,Competence_Probability
count,165831.00,141182.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00,307154.00
mean,-0.38,359.68,0.97,0.03,0.94,0.06,6.81,656.79,45.36,5.47,2.60,712.05,54.60,5.37,2.82,528.39,40.86,4.97,2.09,840.04,59.04,5.74,3.32,1369.08,5.42,1432.49,6088.90,8502.33,0.25,0.37,0.38,0.64,0.20,0.15,1.11,0.95,1846.01,2681.76,2238.91,3219.13,0.62,0.46,0.45,0.08,17.67,15.81,114.11,95.86,0.31,613.29,316.26,4089.23,3387.65,21.97,0.97,0.03,0.94,0.06,0.24,0.33,0.48,0.45
std,1.63,284.66,0.17,0.17,0.23,0.23,2.88,1047.82,19.49,8.72,4.15,1235.51,19.52,9.32,4.89,796.79,10.12,7.49,3.15,1437.93,9.98,9.83,5.69,2216.35,8.77,768.93,1253.12,3997.39,0.44,0.48,0.49,0.48,0.40,0.36,0.78,0.60,974.24,1138.22,1110.80,1207.73,0.64,0.50,0.50,0.28,16.45,12.77,107.75,90.37,0.53,524.43,147.74,3623.96,3021.22,34.37,0.17,0.17,0.23,0.23,0.43,0.37,0.50,0.38
min,-1.00,1.00,0.00,0.00,0.00,0.00,1.00,7.00,12.50,0.06,0.03,21.00,15.63,0.16,0.08,15.00,18.94,0.14,0.06,13.00,44.44,0.09,0.05,29.00,0.11,150.00,3955.00,2351.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,595.24,662.20,721.04,839.94,0.00,0.00,0.00,0.00,3.00,2.00,6.00,1.00,0.00,4.00,4.00,31.00,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01
25%,-1.00,111.00,1.00,0.00,1.00,0.00,4.00,87.00,27.59,0.72,0.34,189.00,34.87,1.43,0.75,95.00,32.76,0.89,0.38,195.00,51.18,1.33,0.77,290.00,1.15,1064.00,3955.00,3778.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1092.89,1854.93,1627.74,1936.98,0.00,0.00,0.00,0.00,7.00,7.00,45.00,38.00,0.00,353.00,213.00,2289.00,1891.00,0.00,1.00,0.00,1.00,0.00,0.00,0.02,0.00,0.04
50%,-1.00,291.00,1.00,0.00,1.00,0.00,7.00,226.00,43.13,1.88,0.89,210.00,56.87,1.58,0.83,205.00,41.67,1.93,0.81,288.00,56.82,1.97,1.14,398.00,1.57,1411.00,6664.00,11464.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,1.00,1366.56,2843.37,1938.77,3571.64,1.00,0.00,0.00,0.00,13.00,12.00,84

In [29]:
df_manual.head()


,Job ID,Job Description spacy_sentencized,Warmth,Competence,Dutch Requirement in Sentence,English Requirement in Sentence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Search Keyword,Platform,Job Title,Company Name,Location,Industry,Dutch Requirement in Job Ad,English Requirement in Job Ad,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Sector Code,Sector,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Sector Job Advertisement Count,Sector Gender Designation Job Advertisement Count,Sector Age Designation Job Advertisement Count,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized_lower,Job Description spacy_tokenized,Job Description spacy_sentencized_cleaned,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description nltk_tokenized,Job Description gensim_tokenized,Job Description bert_tokenized,Warmth_predicted,Warmth_Probability_predicted,Competence_predicted,Competence_Probability_predicted
0,3768944208,Were Happeo a diverse team of passionate prob...,1,1,No,No,1,0,1,0,social work activity,Glassdoor,Inbound Marketer,Happeo,Amsterdam,Ondernemingssoftware en netwerkoplossingen,No,No,1,0,1,0,Q,Health and social work activities,11.00,1208.00,84.30,10.05,4.78,224.00,15.63,1.69,0.89,Female,661.00,46.13,6.22,2.62,770.00,53.73,5.26,3.05,Mixed Age,1433.00,5.67,1362,3955,11464,1,0,0,1,0,0,0,1,3888.45,4529.66,721.04,839.94,2,0,0,1,were happeo a diverse team of passionate prob...,"[were, happeo, a, diverse, team, of, passionat...",were happeo a diverse team of passionate probl...,22,21,153,128,1,"[happeo, diverse, team, passionate, problem-so...","[happeo, divers, team, passion, problem, solve...","[were, ha, ##ppe, ##o, a, diverse, team, of, p...",1,0.97,1,0.76
1,3768944208,Work closely with our Sales and Product leader...,1,1,No,No,1,0,1,0,social work activity,Glassdoor,Inbound Marketer,Happeo,Amsterdam,Ondernemingssoftware en netwerkoplossingen,No,No,1,0,1,0,Q,Health and social work activities,11.00,1208.00,84.30,10.05,4.78,224.00,15.63,1.69,0.89,Female,661.00,46.13,6.22,2.62,770.00,53.73,5.26,3.05,Mixed Age,1433.00,5.67,1362,3955,11464,1,0,0,1,0,0,0,1,3888.45,4529.66,721.04,839.94,2,0,0,1,work closely with our sales and product leader...,"[work, closely, with, our, sales, and, product...",work closely with our sales and product leader...,23,21,140,116,0,"[work, closely, sales, product, leadership, pi...","[work, close, sale, product, leadership, pinpo...","[work, closely, with, our, sales, and, product...",1,0.92,1,0.91
2,3768944208,Assist Marketing & Product to position our pro...,1,1,No,No,1,0,1,0,social work activity,Glassdoor,Inbound Marketer,Happeo,Amsterdam,Ondernemingssoftware en netwerkoplossingen,No,No,1,0,1,0,Q,Health and social work activities,11.00,1208.00,84.30,10.05,4.78,224.00,15.63,1.69,0.89,Female,661.00,46.13,6.22,2.62,770.00,53.73,5.26,3.05,Mixed Age,1433.00,5.67,1362,3955,11464,1,0,0,1,0,0,0,1,3888.45,4529.66,721.04,839.94,2,0,0,1,assist market

In [30]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5358 entries, 0 to 5357
Data columns (total 79 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   Job ID                                                              5358 non-null   object 
 1   Job Description spacy_sentencized                                   5358 non-null   object 
 2   Warmth                                                              5358 non-null   int64  
 3   Competence                                                          5358 non-null   int64  
 4   Dutch Requirement in Sentence                                       5358 non-null   object 
 5   English Requirement in Sentence                                     5358 non-null   object 
 6   Dutch Requirement in Sentence_No                                    5358 non-null   int64  
 7   Dutch Requireme

In [31]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5358 entries, 0 to 5357
Data columns (total 79 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   Job ID                                                              5358 non-null   object 
 1   Job Description spacy_sentencized                                   5358 non-null   object 
 2   Warmth                                                              5358 non-null   int64  
 3   Competence                                                          5358 non-null   int64  
 4   Dutch Requirement in Sentence                                       5358 non-null   object 
 5   English Requirement in Sentence                                     5358 non-null   object 
 6   Dutch Requirement in Sentence_No                                    5358 non-null   int64  
 7   Dutch Requireme

In [32]:
df_manual[['Job ID', 'Job Description spacy_sentencized']].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5358 entries, 0 to 5357
Data columns (total 2 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Job ID                             5358 non-null   object
 1   Job Description spacy_sentencized  5358 non-null   object
dtypes: object(2)
memory usage: 83.8+ KB


In [33]:
df_manual[['Job ID', 'Job Description spacy_sentencized']].head()


,Job ID,Job Description spacy_sentencized
0,3768944208,Were Happeo a diverse team of passionate prob...
1,3768944208,Work closely with our Sales and Product leader...
2,3768944208,Assist Marketing & Product to position our pro...
3,3768944208,Youre not scared of a real scale-up environmen...
4,3768944208,You will be joining a company with highly skil...


In [34]:
df_manual = df_manual.drop_duplicates(subset=['Job ID', 'Job Description spacy_sentencized']).reset_index(drop=True)
df_manual = df_manual.dropna(subset=['Job ID', 'Job Description spacy_sentencized'] + dvs_all_predicted).reset_index(drop=True)
df_manual = df_manual.loc[
    (df_manual['Job Description spacy_sentencized'].progress_apply(lambda x: len(x) > 5))
    & (df_manual['Job Description spacy_sentencized_num_words'] > 2)
    & (df_manual['Job Description spacy_sentencized_num_chars'] > 5)
].reset_index(drop=True)


In [35]:
df_manual['Job Description spacy_sentencized_num_words'].min()


3

In [36]:
df_manual['Job Description spacy_sentencized_num_words'].max()


131

In [37]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5084 entries, 0 to 5083
Data columns (total 79 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   Job ID                                                              5084 non-null   object 
 1   Job Description spacy_sentencized                                   5084 non-null   object 
 2   Warmth                                                              5084 non-null   int64  
 3   Competence                                                          5084 non-null   int64  
 4   Dutch Requirement in Sentence                                       5084 non-null   object 
 5   English Requirement in Sentence                                     5084 non-null   object 
 6   Dutch Requirement in Sentence_No                                    5084 non-null   int64  
 7   Dutch Requireme

In [38]:
df_manual.describe()


,Warmth,Competence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Sector_n,% Sector per Workforce,Sector Job Advertisement Count,Sector Gender Designation Job Advertisement Count,Sector Age Designation Job Advertisement Count,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Warmth_predicted,Warmth_Probability_predicted,Competence_predicted,Competence_Probability_predicted
count,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00
mean,0.30,0.51,1.00,0.00,1.00,0.00,0.97,0.03,0.91,0.09,7.16,562.73,45.18,4.68,2.23,584.94,54.80,4.41,2.31,441.11,38.25,4.15,1.75,705.29,61.51,4.82,2.79,1147.82,4.54,1144.57,6293.51,5737.70,0.16,0.20,0.64,0.35,0.12,0.53,1.04,1.41,1734.33,2773.34,2089.61,3376.41,1.04,0.00,0.96,0.04,15.05,13.96,98.92,82.63,0.74,0.28,0.36,0.51,0.47
std,0.46,0.50,0.03,0.03,0.04,0.04,0.18,0.18,0.28,0.28,2.40,764.54,16.14,6.36,3.03,705.36,16.17,5.32,2.79,560.63,8.83,5.27,2.22,836.87,8.90,5.72,3.31,1370.80,5.42,561.20,1026.02,4257.23,0.37,0.40,0.48,0.48,0.32,0.50,0.60,0.69,849.47,915.33,908.44,1033.09,0.20,0.00,0.20,0.20,10.26,8.58,69.82,58.27,0.65,0.45,0.38,0.50,0.40
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,7.00,19.44,0.06,0.03,21.00,15.63,0.16,0.08,15.00,18.94,0.14,0.06,13.00,44.44,0.09,0.05,29.00,0.11,150.00,3955.00,2351.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,903.69,864.20,721.04,839.94,1.00,0.00,0.00,0.00,3.00,3.00,8.00,6.00,0.00,0.00,0.00,0.00,0.01
25%,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,6.00,226.00,42.75,1.88,0.89,224.00,57.25,1.69,0.89,253.00,31.96,2.38,1.00,288.00,53.14,1.97,1.14,542.00,2.14,1064.00,6664.00,2351.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1366.56,2843.37,1829.74,3246.14,1.00,0.00,1.00,0.00,8.00,8.00,50.00,43.00,0.00,0.00,0.02,0.00,0.03
50%,0.00,1.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,8.00,416.00,42.75,3.46,1.65,557.00,57.25,4.20,2.20,311.00,31.96,2.93,1.23,660.00,67.83,4.51,2.61,973.00,3.85,1064.00,6664.00,2351.00,0.00,0.00,1.00,0.00,0.00,1.00,1.00,2.00,1366.56,2900.09,1829.74,3883.05,1.00,0.00,1.00,0.00,13.00,13.00,85.00,71.00,1.00,0.00,0.15,1.00,0.60
75%,1.00,1.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,8.00,416.00,42.75,3.46,1.65,557.00,57.25,4.20,2.20,311.00,46.68,2.93,1.23,660.00,67.83,4.51,2.61,973.00,3.85,1362.00,6664.00,11464.00,0.00,0.00,1.00,1.00,0.00,1.00,1.00,2.00,1470.63,2900.09,1938.77,3883.05,1.00,0.00,1.00,0.00,20.00,18.00,130.00,109.00,1.00,1.00,0.79,1.00,0.88
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,11.

In [39]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame) and all(col in df_jobs.columns for col in dvs_all), f'ERORR: LENGTH OF DF = {len(df_jobs)}'

with open(f'{data_dir}df_jobs_classified_len.txt', 'w') as f:
    f.write(str(len(df_jobs)))

df_jobs.to_pickle(f'{df_save_dir}df_jobs_classified.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_classified.csv', index=False)


In [40]:
with open(f'{data_dir}df_jobs_classified_len.txt', 'r') as f:
    df_jobs_len = int(f.read())

df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_classified.pkl')
assert len(df_jobs) == df_jobs_len and all(col in df_jobs.columns for col in dvs_all), f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'


In [41]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame) and all(col in df_manual.columns for col in dvs_all_predicted), f'ERORR: LENGTH OF DF = {len(df_manual)}'

with open(f'{data_dir}df_manual_classified_len.txt', 'w') as f:
    f.write(str(len(df_manual)))

df_manual.to_pickle(f'{df_save_dir}df_manual_classified.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_classified.csv', index=False)


In [42]:
with open(f'{data_dir}df_manual_classified_len.txt', 'r') as f:
    df_manual_len = int(f.read())

df_manual = pd.read_pickle(f'{df_save_dir}df_manual_classified.pkl')
assert len(df_manual) == df_manual_len and all(col in df_manual.columns for col in dvs_all_predicted), f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'


## Merge df_jobs with df_manual

In [43]:
merge_on_cols_list = ['Job ID', 'Job Description spacy_sentencized', 'Search Keyword', 'Platform', 'Job Title', 'Company Name', 'Location']
fill_on_cols_list = ['Job Description', 'Industry', 'Rating', 'Employment Type', 'Company URL', 'Job URL', 'Job Age', 'Job Age Number', 'Collection Date', 'Data Row', 'Tracking ID', 'Job Date', 'Type of ownership', 'Language', 'Job Description_num_words', 'Job Description_num_unique_words', 'Job Description_num_chars', 'Job Description_num_chars_no_whitespact_and_punt', 'Job Description_num_punctuations']


In [44]:
df_jobs = pd.merge(
    df_jobs,
    df_manual,
    how='outer', on=merge_on_cols_list, suffixes=('', '_actual')
).drop_duplicates(
    subset=merge_on_cols_list
).reset_index(drop=True)


In [45]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 167 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(119), object(46)
memory usage: 389.0+ MB


In [46]:
df_jobs.isna().sum()


Search Keyword                                                                    0
Platform                                                                          0
Job ID                                                                            0
Job Title                                                                         0
Company Name                                                                      5
Location                                                                          0
Job Description                                                                1429
Rating                                                                       142752
Employment Type                                                               27531
Company URL                                                                    8022
Job URL                                                                        1429
Job Age                                                                     

In [47]:
def fill_contradicting_cols(df_jobs):

    for col in dvs:
        fill_contradicting_dict = {
            col: f'{col}_actual',
            f'{col}_Probability': f'{col}_Probability_predicted'
        }
        for main_col, fill_col in fill_contradicting_dict.items():
            print(f'Filling {main_col} with values from {fill_col}')
            df_jobs[main_col] = df_jobs[main_col].fillna(df_jobs[fill_col])
            mask = (
                (df_jobs[main_col] != df_jobs[fill_col])
                & (~df_jobs[fill_col].isna())
                | (df_jobs[main_col].isna())
            )
            df_jobs.loc[mask, main_col] = df_jobs.loc[mask, fill_col]
            assert df_jobs[main_col].isna().sum() == 0, f'Missing values found in {main_col} column'

    return df_jobs


In [48]:
df_jobs[
    [
        'Warmth', 'Warmth_Probability', 'Warmth_actual', 'Warmth_predicted', 'Warmth_Probability_predicted',
        'Competence', 'Competence_Probability', 'Competence_actual', 'Competence_predicted', 'Competence_Probability_predicted'
    ]
].head()


,Warmth,Warmth_Probability,Warmth_actual,Warmth_predicted,Warmth_Probability_predicted,Competence,Competence_Probability,Competence_actual,Competence_predicted,Competence_Probability_predicted
0,0.00,0.01,NaN,NaN,NaN,0.00,0.02,NaN,NaN,NaN
1,0.00,0.01,NaN,NaN,NaN,0.00,0.06,NaN,NaN,NaN
2,0.00,0.29,NaN,NaN,NaN,1.00,0.64,NaN,NaN,NaN
3,0.00,0.16,NaN,NaN,NaN,1.00,0.92,NaN,NaN,NaN
4,1.00,0.85,NaN,NaN,NaN,0.00,0.06,NaN,NaN,NaN


In [49]:
df_jobs[
    [
        'Warmth', 'Warmth_Probability', 'Warmth_actual', 'Warmth_predicted', 'Warmth_Probability_predicted',
        'Competence', 'Competence_Probability', 'Competence_actual', 'Competence_predicted', 'Competence_Probability_predicted'
    ]
].tail()


,Warmth,Warmth_Probability,Warmth_actual,Warmth_predicted,Warmth_Probability_predicted,Competence,Competence_Probability,Competence_actual,Competence_predicted,Competence_Probability_predicted
308578,NaN,NaN,0.00,0.00,0.08,NaN,NaN,0.00,0.00,0.02
308579,NaN,NaN,0.00,0.00,0.03,NaN,NaN,0.00,0.00,0.02
308580,NaN,NaN,0.00,0.00,0.00,NaN,NaN,0.00,0.00,0.01
308581,NaN,NaN,0.00,0.00,0.04,NaN,NaN,0.00,0.00,0.01
308582,NaN,NaN,0.00,0.00,0.00,NaN,NaN,0.00,0.00,0.01


In [50]:
df_jobs = fill_contradicting_cols(df_jobs)


Filling Warmth with values from Warmth_actual
Filling Warmth_Probability with values from Warmth_Probability_predicted
Filling Competence with values from Competence_actual
Filling Competence_Probability with values from Competence_Probability_predicted


In [51]:
df_jobs[
    [
        'Warmth', 'Warmth_Probability', 'Warmth_actual', 'Warmth_predicted', 'Warmth_Probability_predicted',
        'Competence', 'Competence_Probability', 'Competence_actual', 'Competence_predicted', 'Competence_Probability_predicted'
    ]
].head()


,Warmth,Warmth_Probability,Warmth_actual,Warmth_predicted,Warmth_Probability_predicted,Competence,Competence_Probability,Competence_actual,Competence_predicted,Competence_Probability_predicted
0,0.00,0.01,NaN,NaN,NaN,0.00,0.02,NaN,NaN,NaN
1,0.00,0.01,NaN,NaN,NaN,0.00,0.06,NaN,NaN,NaN
2,0.00,0.29,NaN,NaN,NaN,1.00,0.64,NaN,NaN,NaN
3,0.00,0.16,NaN,NaN,NaN,1.00,0.92,NaN,NaN,NaN
4,1.00,0.85,NaN,NaN,NaN,0.00,0.06,NaN,NaN,NaN


In [52]:
df_jobs[
    [
        'Warmth', 'Warmth_Probability', 'Warmth_actual', 'Warmth_predicted', 'Warmth_Probability_predicted',
        'Competence', 'Competence_Probability', 'Competence_actual', 'Competence_predicted', 'Competence_Probability_predicted'
    ]
].tail()


,Warmth,Warmth_Probability,Warmth_actual,Warmth_predicted,Warmth_Probability_predicted,Competence,Competence_Probability,Competence_actual,Competence_predicted,Competence_Probability_predicted
308578,0.00,0.08,0.00,0.00,0.08,0.00,0.02,0.00,0.00,0.02
308579,0.00,0.03,0.00,0.00,0.03,0.00,0.02,0.00,0.00,0.02
308580,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01
308581,0.00,0.04,0.00,0.00,0.04,0.00,0.01,0.00,0.00,0.01
308582,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01


In [53]:
# Fill in other nan values from columns with '_actual' suffix from df_manual
for col in df_jobs.columns:
    if '_actual' not in col\
        and '_predicted' not in col\
            and '_Probability' not in col\
                and col not in dvs\
                    and col not in merge_on_cols_list + fill_on_cols_list\
                        and df_jobs[col].isna().sum() != 0:
                        df_jobs[col] = df_jobs[col].fillna(df_jobs[f'{col}_actual'])


In [54]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 167 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(119), object(46)
memory usage: 389.0+ MB


In [55]:
df_jobs.isna().sum()


Search Keyword                                                                    0
Platform                                                                          0
Job ID                                                                            0
Job Title                                                                         0
Company Name                                                                      5
Location                                                                          0
Job Description                                                                1429
Rating                                                                       142752
Employment Type                                                               27531
Company URL                                                                    8022
Job URL                                                                        1429
Job Age                                                                     

In [56]:
df_jobs = df_jobs.drop(
    columns=
    [
        col
        for col in df_jobs.columns
        if '_actual' in col
        and col not in ['Warmth_actual', 'Competence_actual']
    ]
).reset_index(drop=True)


In [57]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 101 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(68), object(31)
memory usage: 233.7+ MB


In [58]:
df_jobs.isna().sum()


Search Keyword                                                             0
Platform                                                                   0
Job ID                                                                     0
Job Title                                                                  0
Company Name                                                               5
Location                                                                   0
Job Description                                                         1429
Rating                                                                142752
Employment Type                                                        27531
Company URL                                                             8022
Job URL                                                                 1429
Job Age                                                                 1429
Job Age Number                                                          1429

## Clean df_jobs

In [59]:
%%time
# Conver Job ID and Sentence to str
str_cols = [
    'Job ID',
    'Job Description spacy_sentencized',
]

for col in str_cols:
    df_jobs[col] = df_jobs[col].astype(str, errors='ignore').progress_apply(lambda x: x.strip().replace('[', '').replace(']', ''))
    df_jobs[col] = df_jobs[col].progress_apply(lambda x: x.strip())
    df_jobs[col] = df_jobs[col].progress_apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))
    print(f'{col} converted to str.' if all(df_jobs[col].progress_apply(lambda x: isinstance(x, str))) else f'{col} NOT converted to str.')


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Job ID converted to str.


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Job Description spacy_sentencized converted to str.
CPU times: user 1.22 s, sys: 35.3 ms, total: 1.26 s
Wall time: 1.36 s


In [60]:
# Convert Warmth and Competence to int
# Warmth 1 = 1741, Competence 1 = 3043
int_cols = [
    'Warmth',
    'Competence',
    'Warmth_actual',
    'Competence_actual',
    'Warmth_predicted',
    'Competence_predicted',
]

for col in int_cols:
    df_jobs[col] = df_jobs[col].astype(np.int64, errors='ignore')
    print(f'{col} converted to int.' if all(df_jobs[col].progress_apply(lambda x: isinstance(x, int))) else f'{col} NOT converted to int.')
    print(f'{col} value counts:\n{df_jobs[col].value_counts()}')


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Warmth converted to int.
Warmth value counts:
Warmth
0    234366
1     74217
Name: count, dtype: int64


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Competence converted to int.
Competence value counts:
Competence
0    161987
1    146596
Name: count, dtype: int64


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

...

Competence_actual NOT converted to int.
Competence_actual value counts:
Competence_actual
1.00    2576
0.00    2508
Name: count, dtype: int64


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Warmth_predicted NOT converted to int.
Warmth_predicted value counts:
Warmth_predicted
0.00    3684
1.00    1400
Name: count, dtype: int64


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Competence_predicted NOT converted to int.
Competence_predicted value counts:
Competence_predicted
1.00    2618
0.00    2466
Name: count, dtype: int64


In [61]:
# Convert Warmth_Probability and Competence_Probability to float
# Warmth 1 = 1741, Competence 1 = 3043
float_cols = [
    'Warmth_Probability',
    'Competence_Probability',
    'Warmth_Probability_predicted',
    'Competence_Probability_predicted',
]

for col in float_cols:
    df_jobs[col] = df_jobs[col].astype(np.float64, errors='ignore')
    print(f'{col} converted to float.' if all(df_jobs[col].progress_apply(lambda x: isinstance(x, float))) else f'{col} NOT converted to int.')


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Warmth_Probability converted to float.


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Competence_Probability converted to float.


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Warmth_Probability_predicted converted to float.


progress-bar:   0%|          | 0/308583 [00:00<?, ?it/s]

Competence_Probability_predicted converted to float.


In [62]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 101 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(66), int64(2), object(31)
memory usage: 233.7+ MB


In [63]:
df_jobs = df_jobs.dropna(
    subset=[
        col
        for col in df_jobs.columns
        if '_actual' not in col
        and '_predicted' not in col
        and col not in merge_on_cols_list + fill_on_cols_list
    ]
)


In [64]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 101 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(66), int64(2), object(31)
memory usage: 233.7+ MB


In [65]:
df_jobs = df_jobs.drop_duplicates(subset=merge_on_cols_list + fill_on_cols_list)


In [66]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 101 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(66), int64(2), object(31)
memory usage: 233.7+ MB


In [67]:
df_jobs.isna().sum()


Search Keyword                                                             0
Platform                                                                   0
Job ID                                                                     0
Job Title                                                                  0
Company Name                                                               5
Location                                                                   0
Job Description                                                         1429
Rating                                                                142752
Employment Type                                                        27531
Company URL                                                             8022
Job URL                                                                 1429
Job Age                                                                 1429
Job Age Number                                                          1429

In [68]:
df_jobs.describe()


,Rating,Data Row,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Sector_n,% Sector per Workforce,Sector Job Advertisement Count,Sector Gender Designation Job Advertisement Count,Sector Age Designation Job Advertisement Count,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description_num_words,Job Description_num_unique_words,Job Description_num_chars,Job Description_num_chars_no_whitespact_and_punt,Job Description_num_punctuations,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Warmth,Warmth_Probability,Competence,Competence_Probability,Warmth_actual,Competence_actual,Warmth_predicted,Warmth_Probability_predicted,Competence_predicted,Competence_Probability_predicted
count,165831.00,141182.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,307154.00,307154.00,307154.00,307154.00,307154.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,308583.00,5084.00,5084.00,5084.00,5084.00,5084.00,5084.00
mean,-0.38,359.68,0.97,0.03,0.94,0.06,6.81,656.50,45.36,5.46,2.60,711.44,54.59,5.37,2.82,528.04,40.85,4.97,2.09,839.49,59.05,5.74,3.32,1368.18,5.41,1431.15,6089.72,8489.37,0.25,0.37,0.38,0.64,0.20,0.15,1.11,0.95,1845.63,2682.44,2237.95,3219.73,0.63,0.46,0.46,0.08,17.66,15.81,114.11,95.85,0.31,613.29,316.26,4089.23,3387.65,21.97,0.97,0.03,0.94,0.06,0.24,0.33,0.48,0.45,0.30,0.51,0.28,0.36,0.51,0.47
std,1.63,284.66,0.17,0.17,0.23,0.23,2.88,1046.89,19.48,8.71,4.14,1233.63,19.51,9.31,4.88,795.99,10.11,7.49,3.15,1435.83,9.98,9.81,5.68,2213.40,8.76,768.40,1252.28,4003.11,0.44,0.48,0.49,0.48,0.40,0.36,0.78,0.60,973.83,1137.29,1109.91,1207.15,0.64,0.50,0.50,0.28,16.44,12.76,107.68,90.30,0.54,524.43,147.74,3623.96,3021.22,34.37,0.17,0.17,0.23,0.23,0.43,0.37,0.50,0.38,0.46,0.50,0.45,0.38,0.50,0.40
min,-1.00,1.00,0.00,0.00,0.00,0.00,1.00,7.00,12.50,0.06,0.03,21.00,15.63,0.16,0.08,15.00,18.94,0.14,0.06,13.00,44.44,0.09,0.05,29.00,0.11,150.00,3955.00,2351.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,595.24,662.20,721.04,839.94,0.00,0.00,0.00,0.00,3.00,2.00,6.00,1.00,0.00,4.00,4.00,31.00,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01
25%,-1.00,111.00,1.00,0.00,1.00,0.00,4.00,87.00,27.59,0.72,0.34,189.00,34.87,1.43,0.75,95.00,32.76,0.89,0.38,195.00,51.18,1.33,0.77,290.00,1.15,1064.00,3955.00,3778.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1092.89,1854.93,1627.74,1936.98,0.00,0.00,0.00,0.00,7.00,7.00,45.00,38.00,0.00,353.00,213.00,2289.00,1891.00,0.00,1.00,0.00,1.00,0.00,0.00,0.02,0.00,0.04,0.00,0.

In [69]:
df_jobs[
    [
        'Warmth', 'Warmth_Probability', 'Warmth_actual', 'Warmth_predicted', 'Warmth_Probability_predicted',
        'Competence', 'Competence_Probability', 'Competence_actual', 'Competence_predicted', 'Competence_Probability_predicted',
    ]
].head()


,Warmth,Warmth_Probability,Warmth_actual,Warmth_predicted,Warmth_Probability_predicted,Competence,Competence_Probability,Competence_actual,Competence_predicted,Competence_Probability_predicted
0,0,0.01,NaN,NaN,NaN,0,0.02,NaN,NaN,NaN
1,0,0.01,NaN,NaN,NaN,0,0.06,NaN,NaN,NaN
2,0,0.29,NaN,NaN,NaN,1,0.64,NaN,NaN,NaN
3,0,0.16,NaN,NaN,NaN,1,0.92,NaN,NaN,NaN
4,1,0.85,NaN,NaN,NaN,0,0.06,NaN,NaN,NaN


In [70]:
df_jobs[
    [
        'Warmth', 'Warmth_Probability', 'Warmth_actual', 'Warmth_predicted', 'Warmth_Probability_predicted',
        'Competence', 'Competence_Probability', 'Competence_actual', 'Competence_predicted', 'Competence_Probability_predicted',
    ]
].describe()



,Warmth,Warmth_Probability,Warmth_actual,Warmth_predicted,Warmth_Probability_predicted,Competence,Competence_Probability,Competence_actual,Competence_predicted,Competence_Probability_predicted
count,308583.00,308583.00,5084.00,5084.00,5084.00,308583.00,308583.00,5084.00,5084.00,5084.00
mean,0.24,0.33,0.30,0.28,0.36,0.48,0.45,0.51,0.51,0.47
std,0.43,0.37,0.46,0.45,0.38,0.50,0.38,0.50,0.50,0.40
min,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01
25%,0.00,0.02,0.00,0.00,0.02,0.00,0.04,0.00,0.00,0.03
50%,0.00,0.11,0.00,0.00,0.15,0.00,0.45,1.00,1.00,0.60
75%,0.00,0.70,1.00,1.00,0.79,1.00,0.85,1.00,1.00,0.88
max,1.00,0.99,1.00,1.00,0.99,1.00,0.97,1.00,1.00,0.97


In [71]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_for_analysis.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_for_analysis.csv', index=False)


In [72]:
print(f'Saving classified df_jobs length {len(df_jobs)} to txt file.')
with open(f'{data_dir}df_jobs_for_analysis_len.txt', 'w') as f:
    f.write(str(len(df_jobs)))


Saving classified df_jobs length 308583 to txt file.


In [73]:
get_df_info(
    df_jobs,
    ivs_all=analysis_columns + [f'{col}_actual' for col in analysis_columns] + [f'{col}_predicted' for col in analysis_columns]
)



DF INFO:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 101 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(66), int64(2), object(31)
memory usage: 233.7+ MB
Warmth:
--------------------
Warmth Counts:
Warmth
0    234366
1     74217
Name: count, dtype: int64
--------------------
Warmth Percentages:
Warmth
0   75.90
1   24.10
Name: proportion, dtype: float64
--------------------
Min Warmth value: 0.0
Max Warmth value: 1.0
--------------------
Warmth Mean: 0.24
--------------------
Warmth Standard Deviation: 0.43
Competence:
--------------------
Competence Counts:
Competence
0    161987
1    146596
Name: count, dtype: int64
--------------------
Competence Percentages:
Competence
0   52.50
1   47.50
Name: proportion, dtype: float64
--------------------
Min Competence value: 0.0
Max Competence value: 1.0
--------------------
Competence Mean: 0.48
--------------------
Competence Standard Deviation: 0.5
Warmth

In [74]:
get_df_info(
    df_jobs,
    ivs_all=classified_columns + [f'{col}_predicted' for col in classified_columns ]
)



DF INFO:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 101 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(66), int64(2), object(31)
memory usage: 233.7+ MB
Warmth_Probability:
--------------------
Min Warmth_Probability value: 0.0
Max Warmth_Probability value: 0.994
--------------------
Warmth_Probability Mean: 0.33
--------------------
Warmth_Probability Standard Deviation: 0.37
Competence_Probability:
--------------------
Min Competence_Probability value: 0.011
Max Competence_Probability value: 0.967
--------------------
Competence_Probability Mean: 0.45
--------------------
Competence_Probability Standard Deviation: 0.38
Warmth_Probability_predicted:
--------------------
Min Warmth_Probability_predicted value: 0.0
Max Warmth_Probability_predicted value: 0.992
--------------------
Warmth_Probability_predicted Mean: 0.36
--------------------
Warmth_Probability_predicted Standard Deviation: 0.38
Compet

In [75]:
get_df_info(
    df_jobs,
    ivs_all=dvs_all + [f'{col}_actual' for col in dvs_all if '_Probability' not in col] + [f'{col}_predicted' for col in dvs_all]
)



DF INFO:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308583 entries, 0 to 308582
Columns: 101 entries, Search Keyword to Competence_Probability_predicted
dtypes: category(2), float64(66), int64(2), object(31)
memory usage: 233.7+ MB
Warmth:
--------------------
Warmth Counts:
Warmth
0    234366
1     74217
Name: count, dtype: int64
--------------------
Warmth Percentages:
Warmth
0   75.90
1   24.10
Name: proportion, dtype: float64
--------------------
Min Warmth value: 0.0
Max Warmth value: 1.0
--------------------
Warmth Mean: 0.24
--------------------
Warmth Standard Deviation: 0.43
Competence:
--------------------
Competence Counts:
Competence
0    161987
1    146596
Name: count, dtype: int64
--------------------
Competence Percentages:
Competence
0   52.50
1   47.50
Name: proportion, dtype: float64
--------------------
Min Competence value: 0.0
Max Competence value: 1.0
--------------------
Competence Mean: 0.48
--------------------
Competence Standard Deviation: 0.5
Warmth

### Save dataframe


In [76]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame) and all(col in df_jobs.columns for col in dvs_all), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_for_analysis.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_for_analysis.csv', index=False)


In [77]:
print(f'Saving classified df_jobs length {len(df_jobs)} to txt file.')
with open(f'{data_dir}df_jobs_for_analysis_len.txt', 'w') as f:
    f.write(str(len(df_jobs)))


Saving classified df_jobs length 308583 to txt file.


In [78]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame) and all(col in df_manual.columns for col in ['Warmth_predicted', 'Warmth_Probability_predicted', 'Competence_predicted', 'Competence_Probability_predicted']), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_for_analysis.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_for_analysis.csv', index=False)


In [79]:
print(f'Saving classified df_manual length {len(df_manual)} to txt file.')
with open(f'{data_dir}df_manual_for_analysis_len.txt', 'w') as f:
    f.write(str(len(df_manual)))


Saving classified df_manual length 5084 to txt file.
